
# Goal and Enviroment


In this project, I chose to build a verison II (see Readme) reinforcement learning (RL) agent that controls a robotic arm within Unity's [Reacher](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Learning-Environment-Examples.md#reacher) environment. The goal is to get 20 different robotic arms to maintain contact with the green spheres.

A reward of +0.1 is provided for each timestep that the agent's hand is in the goal location. In order to solve the environment, our agent must achieve a score of +30 averaged across all 20 agents for 100 consecutive episodes.

# Learning Algorithm

During the project I have tested two different algos on the target agent. One is Proximal Policy Optimization (PPO) Algorithm, and the other is Deep Deterministic Policy Gradient (DDPG). It turned out that DDPG performs better in this case, as can be seen from follows. 

#### Deep Deterministic Policy Gradient (DDPG)
The algorithm is outlined in [this paper](https://arxiv.org/pdf/1509.02971.pdf), _Continuous Control with Deep Reinforcement Learning_, by researchers at Google Deepmind. In this paper, the authors present "a model-free, off-policy actor-critic algorithm using deep function approximators that can learn policies in high-dimensional, continuous action spaces." They highlight that DDPG can be viewed as an extension of Deep Q-learning to continuous tasks.

I used [ddpg_agent.py](https://github.com/udacity/deep-reinforcement-learning/tree/master/ddpg-pendulum) from Udacity's course Github as a template. I further experimented with the DDPG algorithm based on other concepts covered in Udacity's classroom and lessons. My understanding and implementation of this algorithm (including various customizations) are discussed below.

#### Actor-Critic Method
Actor-critic methods leverage the strengths of both policy-based and value-based methods.

Using a policy-based approach, the agent (actor) learns how to act by directly estimating the optimal policy and maximizing reward through gradient ascent. Meanwhile, employing a value-based approach, the agent (critic) learns how to estimate the value (i.e., the future cumulative reward) of different state-action pairs. Actor-critic methods combine these two approaches in order to accelerate the learning process. Actor-critic agents are also more stable than value-based agents, while requiring fewer training samples than policy-based agents.

You can find the actor-critic logic implemented as part of the `Agent()` class [here](https://github.com/tommytracey/DeepRL-P2-Continuous-Control/blob/master/ddpg_agent.py#L45) in `ddpg_agent.py` of the source code. The actor-critic models can be found via their respective `Actor()` and `Critic()` classes [here](https://github.com/tommytracey/DeepRL-P2-Continuous-Control/blob/master/model.py) in `models.py`.

Note: As we did with Double Q-Learning in the last project, we're again leveraging local and target networks to improve stability. This is where one set of parameters `w` is used to select the best action, and another set of parameters `w'` is used to evaluate that action. In this project, local and target networks are implemented separately for both the actor and the critic.

# Code Implementation

The code used here is adapted from the "ddpg-pendulum" tutorial from the Deep Reinforcement Learning Nanodegree, and has been slightly adjusted for being used with the Reacher environment.

The code consist of :

- model.py : In this python file, a PyTorch QNetwork class is implemented which inherits nn.Module base class. This is a regular fully connected Deep Neural Network using the PyTorch Framework. This network will be trained to predict the action to perform depending on the environment observed states. This Neural Network is used by the DQN agent and is composed of :

- input layer of size 37
2 hidden fully connected layers of 64 cells each
output layer which size depends of the action_size parameter passed in the constructor, which is 4 in our problem




# DDPG Parameters and Results

#### Agent Hyperparameters

The hyperparameters of the agent are listed as below:

```
BUFFER_SIZE = int(1e6)  # replay buffer size
BATCH_SIZE = 256        # minibatch size
GAMMA = 0.98            # discount factor

TAU = 1e-3              # for soft update of target parameters
LR_ACTOR  = 2e-3         # learning rate of the actor 
LR_CRITIC = 1e-4        # learning rate of the critic
WEIGHT_DECAY = 0        # L2 weight decay

EPSILON_DECAY = 2e-6    # decay for epsilon above
NOISE_SIGMA = 0.05      # sigma for Ornstein-Uhlenbeck noise
NOISE_THETA = 0.10

L1_SIZE = 128
L2_SIZE = 64
```

- `GAMMA = 0.99` is the discount factor that controls how far-sighted each agent is with respect to rewards. `GAMMA = 0` implies that only the immediate reward is important and `GAMMA = 1.0` implies that all rewards are equally important, irrespective whether they are realised soon and much later
- `TAU = 0.001` controls the degree to which the target network parameters are adjusted toward those of the local network. `TAU = 0` implies no adjustment (the target network does not ever learn) and `TAU = 1` implies that the target network parameters are completely replaced with the local network parameters
- `LR_ACTOR = 0.0001` is the learning rate for the gradient descent update of the local actor weights
- `LR_CRITIC = 0.001` is the learning rate for the gradient descent update of the local critic weights
- `BUFFER_SIZE = 100000` is the number of experience tuples `(state, action, reward, next_state, done)` that are stored in the replay buffer and avaiable for learning
- `BATCH_SIZE = 128` is the number of tuples that are sampled from the replay buffer for learning
- During training, the predicted actions were corrupted with noise based on an Ornstein-Uhlenbeck process with mean `mu = 0`, mean reversion rate `theta = 0.15` and variance `sigma = 0.2` 


# Plot of Rewards

Given the chosen architecture and parameters, our results are